# NM vs. GD: Multivariate Linear Regression

### 1. Import Dependencies

In [1]:
from sklearn import datasets
import numpy as np
import numpy.linalg as lnp
from time import time

### 2. Problem Setup

#### 2.1 [Iris flower data set] (https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)
We experiment with Newton's Method and Gradient Descent using the Iris Flower Dataset, provided as a built-in dataset in Sklearn. Iris is a multivariate dataset with 3 inference classes of iris flower -- Setosa, Versicolour, and Virginica, as well as 4 dependent features to guide prediction -- Sepal length, Sepal width, Petal length, Petal width. 

We utilise multivariate linear regression, optimised by Gradient Descent and Newton's Method respectively to model to relationship between independent variable $y$ (iris flower class) and dependent variables $x=[x_1, x_2, x_3, x_4]$:


In [13]:
data, valid = datasets.load_iris(), True 
X = data["data"]
Y = data["target"]
m = Y.shape[0]

# print(data["feature_names"])

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [12]:
def MSE(dist):
    global m
    return lnp.norm(dist**2, 1)/(2*m)

# for the bias:
X = np.insert(X, 0, 1, axis=1)
if valid:
    margin = 0.0233 #global minima at 3 sig fig for MSE on this distribution & overflow
else:
    margin = 11 #preliminary

#random init: sample to make better
#np.random.seed(1)
#theta = np.ndarray((X.shape[1]))
theta = np.full((X.shape[1]), 5) #weights
y_hat = np.matmul(X, theta)
dist = Y-y_hat
X_T = X.T
cost = MSE(dist) #no need transpose because np doesn't differentiate between column & row vectors


['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


### 3. Newton's Method in Optimisation

In [3]:
def nm(theta, y_hat, X_T, dist, cost, lr):
    start = time()
    global X, Y, margin, m, indices
    cost_log = []
    epoch = 0
    hessian = np.matmul(X_T, X)
    inv_hessian = lnp.inv(hessian)

    while cost > margin:
        #cost_log.append(cost) #commented for timing
        #print("Epoch #" + str(epoch) + ":", cost)
        epoch += 1
        grad = np.matmul(X_T, dist/-m)
        theta = theta-lr*np.matmul(inv_hessian, grad)

        y_hat = np.matmul(X, theta)
        dist = Y-y_hat
        X_T = X.T
        cost = MSE(dist)
    end = time()
    cost_log.append(cost)
    print("\n" + "Finished NM in " + str(epoch+1), "epochs with error " + str(cost_log[-1]) + "\n")
    print("Optimal theta:", theta)
    print("\n\n" + "y_hat, y")
    for i in indices:
        print(y_hat[i], Y[i])
    return end-start

### 4. Gradient Descent

In [4]:
def gd(theta, y_hat, X_T, dist, cost, lr):
    start = time()
    global X, Y, margin, m, indices
    cost_log = []
    epoch = 0

    while cost > margin:
        #cost_log.append(cost) #commented for timing
        epoch += 1
        grad = np.matmul(X_T, dist)/-m
        theta = theta-lr*grad

        y_hat = np.matmul(X, theta)
        dist = Y-y_hat
        X_T = X.T
        cost = MSE(dist)
    end = time()
    cost_log.append(cost)
    print("\n" + "Finished GD in " + str(epoch+1), "epochs with error " + str(cost_log[-1]) + "\n")
    print("Optimal theta:", theta)

    print("\n\n" + "y_hat, y")
    for i in indices:
        print(y_hat[i], Y[i])
    return end-start

### 5. Sampling & Testing

In [6]:
indices = [np.random.randint(0,m) for i in range(100)] #randomly sample 10 pairs for testing
#print(theta)

GD_time = gd(theta, y_hat, X_T, dist, cost, 0.032) # max 3 dp. cuz overflow
print("\n\n" + "#"*10 + "\n\n")
NM_time = nm(theta, y_hat, X_T, dist, cost, 150)

print("\n\n\n" + "GD time", GD_time, "| NM time", NM_time)
if GD_time > NM_time:
    print("NM is faster")
    #if runtime is to fast, can't always tell cuz of sig fig
else:
    print("GD is faster")


Finished GD in 13430 epochs with error 0.023299961074107964

Optimal theta: [ 0.34672323 -0.13559269 -0.05407266  0.23162183  0.61839075]


y_hat, y
1.1265075222822922 1
-0.061731272562303555 0
0.9271354666962658 1
0.9352874697216145 1
1.3950705658803275 1
-0.08023001233992957 0
-0.11443522828992857 0
1.2508213455115704 1
0.09881462220891538 0
1.419840745672771 1
0.0225261632191371 0
1.2508213455115704 1
1.770503224977595 2
1.4854076742442344 1
-0.038808383256667925 0
0.02648251780110357 0
-0.16350433180021967 0
0.006304365671656667 0
1.2757818831859362 1
0.048937612160696115 0
0.012683955065178956 0
1.221579489762053 1
1.7458641896678622 2
1.5410697201339756 2
-0.10087595941541959 0
0.048937612160696115 0
-0.12623287630939942 0
-0.024265998061008487 0
-0.0020869312151078484 0
0.04992072848184678 0
1.286835708745672 1
-0.08610507313513273 0
0.9465805891427661 1
0.04475275649410215 0
-0.10264837304724596 0
1.7002577049180065 2
1.640699927395939 2
-0.09425707616048155 0
1.28859732960071

### 6. Debugging

In [7]:
"""
SANITY CHECK:
#foo1, foo2, foo3, foo4, foo5, foo6, foo7 = theta, y_hat, cost, Y, m, X, margin #insert before GD
assert(theta.all() == foo1.all())
assert(y_hat.all() == foo2.all())
assert(cost == foo3)
assert(Y.all() == foo4.all())
assert(m == foo5)
assert(X.all() == foo6.all())
assert(margin == foo7)
"""

'\nSANITY CHECK:\n#foo1, foo2, foo3, foo4, foo5, foo6, foo7 = theta, y_hat, cost, Y, m, X, margin #insert before GD\nassert(theta.all() == foo1.all())\nassert(y_hat.all() == foo2.all())\nassert(cost == foo3)\nassert(Y.all() == foo4.all())\nassert(m == foo5)\nassert(X.all() == foo6.all())\nassert(margin == foo7)\n'